In [533]:
import pandas as pd
from elo import Elo

In [534]:
elo = Elo(0.33542897, 0.46681261)

In [535]:
df = pd.concat([pd.read_csv('Super Saturday #2 Scoreboards - Super Saturday 1.csv').assign(Day='Super Saturday 1'),
                pd.read_csv('Super Saturday #2 Scoreboards - Super Saturday 2.csv').assign(Day='Super Saturday 2')])

In [536]:
df.head()

,Round,Round offset,Column offset,Row offset,Board offset,Team 1,Team 1 full name,Team 1 division,Player 1,Rating 1,Team 2,Team 2 full name,Team 2 division,Player 2,Rating 2,Day
0,1,5,0,0,0,Delhi,Delhi Dynamite,Eastern,Salem Saleh,2638,Dallas,Dallas Destiny,Pacific,Jeffery Xiong,2633,Super Saturday 1
1,1,5,0,0,1,Delhi,Delhi Dynamite,Eastern,M R Lalith Babu,2525,Dallas,Dallas Destiny,Pacific,Julio Sadorra,2574,Super Saturday 1
2,1,5,0,0,2,Delhi,Delhi Dynamite,Eastern,Tania Sachdev,2391,Dallas,Dallas Destiny,Pacific,Andrey Gorovets,2522,Super Saturday 1
3,1,5,0,0,3,Delhi,Delhi Dynamite,Eastern,Hemant Sharma,2342,Dallas,Dallas Destiny,Pacific,Artur Safin,2229,Super Saturday 1
4,1,5,1,0,0,Armenia,Armenia Eagles,Eastern,Zaven Andriasyan,2593,Seattle,Seattle Sluggers,Pacific,Hikaru Nakamura,2781,Super Saturday 1


In [537]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 512 entries, 0 to 255
Data columns (total 16 columns):
Round               512 non-null int64
Round offset        512 non-null int64
Column offset       512 non-null int64
Row offset          512 non-null int64
Board offset        512 non-null int64
Team 1              512 non-null object
Team 1 full name    512 non-null object
Team 1 division     512 non-null object
Player 1            512 non-null object
Rating 1            512 non-null int64
Team 2              512 non-null object
Team 2 full name    512 non-null object
Team 2 division     512 non-null object
Player 2            512 non-null object
Rating 2            512 non-null int64
Day                 512 non-null object
dtypes: int64(7), object(9)
memory usage: 68.0+ KB


In [538]:
def score(x):
    # Player 1 is white on board 1 and 3 (board offset 0 and 2)
    p = x['Board offset'] % 2 == 0
    white_elo = x['Rating 1'] if p else x['Rating 2']
    black_elo = x['Rating 2'] if p else x['Rating 1']
    pr = elo.probabilities(white_elo, black_elo)
    return pd.Series({'Player 1 score': (pr['white'] if p else pr['black']) + pr['draw'] / 2,
                      'Player 2 score': (pr['black'] if p else pr['white']) + pr['draw'] / 2})

score_df = pd.concat([df, df.apply(score, axis=1)], axis=1)

In [539]:
pd.concat([score_df[score_df['Player 1'] == 'Fabiano Caruana'],
           score_df[score_df['Player 2'] == 'Fabiano Caruana']])

,Round,Round offset,Column offset,Row offset,Board offset,Team 1,Team 1 full name,Team 1 division,Player 1,Rating 1,Team 2,Team 2 full name,Team 2 division,Player 2,Rating 2,Day,Player 1 score,Player 2 score
36,2,8,1,3,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Armenia,Armenia Eagles,Eastern,Zaven Andriasyan,2593,Super Saturday 1,0.838179,0.161821
108,4,14,3,9,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Cannes,Cannes Blockbusters,Central,Matthieu Cornette,2602,Super Saturday 1,0.831221,0.168779
152,5,17,6,12,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Mumbai,Mumbai Movers,Eastern,B. Adhiban,2643,Super Saturday 1,0.798563,0.201437
208,7,23,4,18,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Ljubljana,Ljubljana Turtles,Central,Luka Lenic,2646,Super Saturday 1,0.795598,0.204402
8,1,5,2,0,0,Reykjavik,Reykjavik Puffins,Central,Johann Hjartarson,2539,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Super Saturday 1,0.238697,0.761303
64,3,11,0,6,0,Delhi,Delhi Dynamite,Eastern,Salem Saleh,2638,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Super Saturday 1,0.376381,0.623619
180,6,20,5,15,0,Marseille,Marseille Migraines,Central,Jean-Marc Degraeve,2561,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Super Saturday 1,0.266642,0.733358
252,8,26,7,21,0,Volga,Volga Stormbringers,Eastern,Azat Sharafiev,2433,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Super Saturday 1,0.132104,0.867896


In [658]:
# Sum score by player (+ day, conference, board, team)
conference = lambda x: {'Atlantic': 'Atlantic & Pacific', 'Eastern': 'Central & Eastern', 'Central': 'Central & Eastern', 'Pacific': 'Atlantic & Pacific'}[x]

total_df = pd.concat([
    score_df.apply(lambda x: pd.Series({'Day': x['Day'], 'Conference': conference(x['Team 1 division']), 'Division': x['Team 1 division'],
                                        'Board offset': x['Board offset'], 'Team': x['Team 1 full name'], 'Player': x['Player 1'],
                                        'Rating': x['Rating 1'], 'Score': x['Player 1 score']}), axis=1),
    score_df.apply(lambda x: pd.Series({'Day': x['Day'], 'Conference': conference(x['Team 2 division']), 'Division': x['Team 2 division'],
                                        'Board offset': x['Board offset'], 'Team': x['Team 2 full name'], 'Player': x['Player 2'],
                                        'Rating': x['Rating 2'], 'Score': x['Player 2 score']}), axis=1)]) \
    .groupby(['Day', 'Conference', 'Division', 'Board offset', 'Team', 'Player']).agg({'Score': 'sum', 'Rating': 'mean'})

In [670]:
total_df.head()

Score  \
Day              Conference         Division Board offset Team                    Player                            
Super Saturday 1 Atlantic & Pacific Atlantic 0            Buenos Aires Krakens    Federico Perez Ponsa   4.006997   
                                                          Montclair Sopranos      Aleksandr Lenderman    3.825211   
                                                          Pittsburgh Pawngrabbers Awonder Liang          3.605271   
                                                          St. Louis Arch Bishops  Fabiano Caruana        6.249736   
                                             1            Buenos Aires Krakens    Andres Carlos Obregon  4.410439   

                                                                                                         Rating  
Day              Conference         Division Board offset Team                    Player                         
Super Saturday 1 Atlantic & Pacific Atlantic 0            Buenos Aires Krakens    Federico Perez Ponsa     2588  
                                                          Montclair Sopranos      Aleksandr Lenderman      2565  
                                                          Pittsburgh Pawngrabbers Awonder Liang            2545  
                                                          St. Louis Arch Bishops  Fabiano Caruana          2799  
                                             1            Buenos Aires Krakens    Andres Carlos Obregon    2514

In [671]:
# Best score by division and board
total_df.groupby(['Day', 'Conference', 'Board offset'], group_keys=False).apply(lambda x: x.sort_values('Score', ascending=False).head(1))

Score  \
Day              Conference         Division Board offset Team                   Player                          
Super Saturday 1 Atlantic & Pacific Atlantic 0            St. Louis Arch Bishops Fabiano Caruana      6.249736   
                                    Pacific  1            Chengdu Pandas         Li Chao              6.306680   
                                    Atlantic 2            St. Louis Arch Bishops Yaroslav Zherebukh   5.952697   
                                             3            Montclair Sopranos     Advait Patel         6.078830   
                 Central & Eastern  Central  0            Ljubljana Turtles      Luka Lenic           3.883455   
                                    Eastern  1            Armenia Eagles         Samvel Ter-Sahakyan  4.079369   
                                             2            Armenia Eagles         Karen Grigoryan      4.684966   
                                    Central  3            Reykjavik Puffins      David Kjartansson    5.717173   
Super Saturday 2 Atlantic & Pacific Atlantic 0            Miami Champions        Eduardo Iturrizaga   4.925186   
                                             1            Webster Windmills      Ray Robson           5.340740   
                                    Pacific  2            San Jose Hackers       Zviad Izoria         5.331049   
                                    Atlantic 3            Minnesota Blizzard     Daniel Gurevich      5.599725   
                 Central & Eastern  Eastern  0            Norway Gnomes          Magnus Carlsen       6.039194   
                                             1            Norway Gnomes          Nils Grandelius      4.931451   
                                             2            Norway Gnomes          Ruifeng Li           4.981693   
                                    Central  3            Amsterdam Mosquitoes   Miguoel Admiraal     5.359886   

                                                                                                      Rating  
Day              Conference         Division Board offset Team                   Player                       
Super Saturday 1 Atlantic & Pacific Atlantic 0            St. Louis Arch Bishops Fabiano Caruana        2799  
                                    Pacific  1            Chengdu Pandas         Li Chao                2743  
                                    Atlantic 2            St. Louis Arch Bishops Yaroslav Zherebukh     2621  
                                             3            Montclair Sopranos     Advait Patel           2430  
                 Central & Eastern  Central  0            Ljubljana Turtles      Luka Lenic             2646  
                                    Eastern  1            Armenia Eagles         Samvel Ter-Sahakyan    2577  
                                             2            Armenia Eagles         Karen Grigoryan        2568  
                                    Central  3            Reykjavik Puffins      David Kjartansson      2385  
Super Saturday 2 Atlantic & Pacific Atlantic 0            Miami Champions        Eduardo Iturrizaga     2665  
                                             1            Webster Windmills      Ray Robson             2644  
                                    Pacific  2            San Jose Hackers       Zviad Izoria           2601  
                                    Atlantic 3            Minnesota Blizzard     Daniel Gurevich        2471  
                 Central & Eastern  Eastern  0            Norway Gnomes          Magnus Carlsen         2827  
                                             1            Norway Gnomes          Nils Grandelius        2653  
                                             2            Norway Gnomes          Ruifeng Li             2585  
                                    Central  3            Amsterdam Mosquitoes   Miguoel Admiraal       2448

In [663]:
# Best team per day
total_df.groupby(level=['Day', 'Team'], group_keys=False).agg({'Score': 'sum', 'Rating': 'mean'}) \
    .groupby(level='Day', group_keys=False).apply(lambda x: x.sort_values('Score', ascending=False).head(1))

,,Score,Rating
Day,Team,,
Super Saturday 1,Chengdu Pandas,21.089945,2560.0
Super Saturday 2,San Diego Surfers,18.245406,2522.5


In [664]:
# Best rated player by division and board
total_df.groupby(['Day', 'Conference', 'Board offset'], group_keys=False).apply(lambda x: x.sort_values('Rating', ascending=False).head(1))

Score  \
Day              Conference         Division Board offset Team                   Player                          
Super Saturday 1 Atlantic & Pacific Atlantic 0            St. Louis Arch Bishops Fabiano Caruana      6.249736   
                                    Pacific  1            Chengdu Pandas         Li Chao              6.306680   
                                    Atlantic 2            St. Louis Arch Bishops Yaroslav Zherebukh   5.952697   
                                             3            Montclair Sopranos     Advait Patel         6.078830   
                 Central & Eastern  Central  0            Ljubljana Turtles      Luka Lenic           3.883455   
                                    Eastern  1            Armenia Eagles         Samvel Ter-Sahakyan  4.079369   
                                             2            Armenia Eagles         Karen Grigoryan      4.684966   
                                    Central  3            Reykjavik Puffins      David Kjartansson    5.717173   
Super Saturday 2 Atlantic & Pacific Pacific  0            San Jose Hackers       Rauf Mamedov         4.882205   
                                    Atlantic 1            Webster Windmills      Ray Robson           5.340740   
                                    Pacific  2            San Jose Hackers       Zviad Izoria         5.331049   
                                    Atlantic 3            Minnesota Blizzard     Daniel Gurevich      5.599725   
                 Central & Eastern  Eastern  0            Norway Gnomes          Magnus Carlsen       6.039194   
                                             1            Norway Gnomes          Nils Grandelius      4.931451   
                                             2            Norway Gnomes          Ruifeng Li           4.981693   
                                    Central  3            Amsterdam Mosquitoes   Miguoel Admiraal     5.359886   

                                                                                                      Rating  
Day              Conference         Division Board offset Team                   Player                       
Super Saturday 1 Atlantic & Pacific Atlantic 0            St. Louis Arch Bishops Fabiano Caruana        2799  
                                    Pacific  1            Chengdu Pandas         Li Chao                2743  
                                    Atlantic 2            St. Louis Arch Bishops Yaroslav Zherebukh     2621  
                                             3            Montclair Sopranos     Advait Patel           2430  
                 Central & Eastern  Central  0            Ljubljana Turtles      Luka Lenic             2646  
                                    Eastern  1            Armenia Eagles         Samvel Ter-Sahakyan    2577  
                                             2            Armenia Eagles         Karen Grigoryan        2568  
                                    Central  3            Reykjavik Puffins      David Kjartansson      2385  
Super Saturday 2 Atlantic & Pacific Pacific  0            San Jose Hackers       Rauf Mamedov           2678  
                                    Atlantic 1            Webster Windmills      Ray Robson             2644  
                                    Pacific  2            San Jose Hackers       Zviad Izoria           2601  
                                    Atlantic 3            Minnesota Blizzard     Daniel Gurevich        2471  
                 Central & Eastern  Eastern  0            Norway Gnomes          Magnus Carlsen         2827  
                                             1            Norway Gnomes          Nils Grandelius        2653  
                                             2            Norway Gnomes          Ruifeng Li             2585  
                                    Central  3            Amsterdam Mosquitoes   Miguoel Admiraal       2448

In [669]:
# Some rating statistics per team
total_df.groupby(level='Team').describe()['Rating'].sort_values('mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Team,,,,,,,,
Chengdu Pandas,4.0,2560.00,270.526647,2173.0,2473.75,2658.5,2744.75,2750.0
Norway Gnomes,4.0,2531.75,329.330002,2062.0,2454.25,2619.0,2696.50,2827.0
San Diego Surfers,4.0,2522.50,141.266887,2333.0,2457.50,2554.5,2619.50,2648.0
Stockholm Snowballs,4.0,2521.75,182.977913,2271.0,2458.50,2557.0,2620.25,2702.0
Webster Windmills,4.0,2520.00,158.431899,2315.0,2435.00,2559.5,2644.50,2646.0
St. Louis Arch Bishops,4.0,2516.25,363.555933,1983.0,2461.50,2641.5,2696.25,2799.0
Seattle Sluggers,4.0,2509.25,248.172487,2179.0,2442.25,2538.5,2605.50,2781.0
Montclair Sopranos,4.0,2499.00,55.263008,2430.0,2479.50,2500.5,2520.00,2565.0
Miami Champions,4.0,2496.00,227.679599,2164.0,2443.00,2577.5,2630.50,2665.0
